In [2]:
%load_ext autoreload
%autoreload 2

import os
import time
import sys

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *
from distributed_utilities import *

Setting environment for AWS compute node


No vtk
File does not exist: /shared/CSHL_data_processed/MD585/MD585_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD585/MD585_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD585/MD585_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD585/MD585_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD589/MD589_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD589/MD589_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD590/MD590_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD590/MD590_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD590/MD590_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD590/MD590_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD591/MD591_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD591/MD591_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD591/MD591_cropbox.txt
File does not exist:

In [6]:
transfer_data_synced('CSHL_data_processed/MD589/MD589_sorted_filenames.txt', from_hostname='s3', to_hostname='ec2', is_dir=False)

rm -rf /shared/CSHL_data_processed/MD589/MD589_sorted_filenames.txt && mkdir -p /shared/CSHL_data_processed/MD589
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD589/MD589_sorted_filenames.txt /shared/CSHL_data_processed/MD589/MD589_sorted_filenames.txt


Child returned 0
Child returned 0


In [7]:
transfer_data_synced('CSHL_data_processed/MD589/MD589_elastix_output', from_hostname='s3', to_hostname='ec2', is_dir=True)

rm -rf /shared/CSHL_data_processed/MD589/MD589_elastix_output && mkdir -p /shared/CSHL_data_processed/MD589
aws s3 cp --recursive s3://mousebrainatlas-data/CSHL_data_processed/MD589/MD589_elastix_output /shared/CSHL_data_processed/MD589/MD589_elastix_output


Child returned 0
Child returned 0


# Align

In [3]:
stack = 'MD589'
_, sections_to_filenames = DataManager.load_sorted_filenames(stack=stack)
filenames = [fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)]

In [41]:
run_distributed("%(script)s %(stack)s %(input_dir)s %(elastix_output_dir)s \'%%(kwargs_str)s\'" % \
                {'stack': stack,
                'script': os.path.join(REPO_DIR, 'preprocess', 'align_consecutive_v2.py'),
                'input_dir': os.path.join(RAW_DATA_DIR, stack),
                'elastix_output_dir': os.path.join(DATA_DIR, stack, stack+'_elastix_output')},
                kwargs_list=[{'prev_fn': filenames[i-1], 'curr_fn': filenames[i]} for i in range(1, len(filenames))],
                argument_type='list',
               cluster_size=12)

/shared/MouseBrainAtlas/preprocess/align_consecutive_v2.py MD589 /shared/CSHL_data/MD589 /shared/CSHL_data_processed/MD589/MD589_elastix_output '[{"curr_fn": "MD589-IHC1-2015.07.30-21.30.07_MD589_1_0001", "prev_fn": "MD589-N1-2015.07.30-16.19.59_MD589_1_0001"}, {"curr_fn": "MD589-N1-2015.07.30-16.19.59_MD589_2_0002", "prev_fn": "MD589-IHC1-2015.07.30-21.30.07_MD589_1_0001"}, {"curr_fn": "MD589-IHC1-2015.07.30-21.30.07_MD589_2_0002", "prev_fn": "MD589-N1-2015.07.30-16.19.59_MD589_2_0002"}, {"curr_fn": "MD589-N1-2015.07.30-16.19.59_MD589_3_0003", "prev_fn": "MD589-IHC1-2015.07.30-21.30.07_MD589_2_0002"}, {"curr_fn": "MD589-IHC1-2015.07.30-21.30.07_MD589_3_0003", "prev_fn": "MD589-N1-2015.07.30-16.19.59_MD589_3_0003"}, {"curr_fn": "MD589-N2-2015.07.30-16.22.26_MD589_1_0004", "prev_fn": "MD589-IHC1-2015.07.30-21.30.07_MD589_3_0003"}, {"curr_fn": "MD589-IHC2-2015.07.30-21.32.04_MD589_1_0004", "prev_fn": "MD589-N2-2015.07.30-16.22.26_MD589_1_0004"}, {"curr_fn": "MD589-N2-2015.07.30-16.22.26_

qsub returned.


# Compose

In [12]:
transfer_data_synced('CSHL_data_processed/MD589/MD589_anchor.txt', from_hostname='s3', to_hostname='ec2', is_dir=False)

rm -rf /shared/CSHL_data_processed/MD589/MD589_anchor.txt && mkdir -p /shared/CSHL_data_processed/MD589
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD589/MD589_anchor.txt /shared/CSHL_data_processed/MD589/MD589_anchor.txt


Child returned 0
Child returned 0


In [4]:
anchor_fn = DataManager.load_anchor_filename(stack=stack)

In [7]:
t = time.time()
print 'Composing transform...'

elastix_output_dir = os.path.join(DATA_DIR, stack, stack + '_elastix_output')
output_fn = os.path.join(elastix_output_dir, '%(stack)s_transformsTo_%(anchor_fn)s.pkl' % \
                                                dict(stack=stack, anchor_fn=anchor_fn))

run_distributed("%(script)s %(stack)s %(elastix_output_dir)s \'%%(kwargs_str)s\' %(anchor_idx)d %(output_fn)s" % \
            {'stack': stack,
            'script': os.path.join(REPO_DIR, 'preprocess', 'compose_transform_thumbnail_v2.py'),
            'elastix_output_dir': elastix_output_dir,
            'anchor_idx': filenames.index(anchor_fn),
            'output_fn': output_fn},
            kwargs_list=[{'filenames': filenames}],
            argument_type='list',
               cluster_size=1)

# transforms_filename = '%(stack)s_transformsTo_%(anchor_fn)s.pkl' % dict(stack=stack, anchor_fn=anchor_fn)
# linked_name = os.path.join(elastix_output_dir, '%(stack)s_transformsTo_anchor.pkl' % dict(stack=stack))
# execute_command('rm ' + linked_name)
# execute_command('ln -s ' + transforms_filename + ' ' + linked_name)

print 'done in', time.time() - t, 'seconds'

Composing transform...
/shared/MouseBrainAtlas/preprocess/compose_transform_thumbnail_v2.py MD589 /shared/CSHL_data_processed/MD589/MD589_elastix_output '[{"filenames": ["MD589-N1-2015.07.30-16.19.59_MD589_1_0001", "MD589-IHC1-2015.07.30-21.30.07_MD589_1_0001", "MD589-N1-2015.07.30-16.19.59_MD589_2_0002", "MD589-IHC1-2015.07.30-21.30.07_MD589_2_0002", "MD589-N1-2015.07.30-16.19.59_MD589_3_0003", "MD589-IHC1-2015.07.30-21.30.07_MD589_3_0003", "MD589-N2-2015.07.30-16.22.26_MD589_1_0004", "MD589-IHC2-2015.07.30-21.32.04_MD589_1_0004", "MD589-N2-2015.07.30-16.22.26_MD589_2_0005", "MD589-IHC2-2015.07.30-21.32.04_MD589_2_0005", "MD589-N2-2015.07.30-16.22.26_MD589_3_0006", "MD589-IHC2-2015.07.30-21.32.04_MD589_3_0006", "MD589-N3-2015.07.30-16.25.24_MD589_1_0007", "MD589-IHC3-2015.07.30-21.34.34_MD589_1_0007", "MD589-N3-2015.07.30-16.25.24_MD589_2_0008", "MD589-IHC3-2015.07.30-21.34.34_MD589_2_0008", "MD589-N3-2015.07.30-16.25.24_MD589_3_0009", "MD589-IHC3-2015.07.30-21.34.34_MD589_3_0009", "M

qsub returned.


# Warp

In [11]:
tb_fmt = 'tif'
pad_bg_color = 'auto'

In [20]:
t = time.time()
print 'Warping...'

transforms_filename = os.path.join(elastix_output_dir,
                    '%(stack)s_transformsTo_%(anchor_fn)s.pkl' % \
                    dict(stack=stack, anchor_fn=anchor_fn))
transforms_to_anchor = pickle.load(open(transforms_filename, 'r'))

input_dir = os.path.join(RAW_DATA_DIR, stack)
out_dir = os.path.join(DATA_DIR, stack, stack + '_thumbnails_alignedTo_' + anchor_fn)
script = os.path.join(REPO_DIR, 'preprocess', 'warp_crop_IM_v2.py')

if pad_bg_color == 'auto':
    run_distributed('%(script)s %(stack)s %(input_dir)s %(out_dir)s %%(transform)s %%(filename)s %%(output_fn)s thumbnail 0 0 2000 1500 %%(pad_bg_color)s' % \
                    {'script': script,
                    'stack': stack,
                    'input_dir': input_dir,
                    'out_dir': out_dir
                    },
                    kwargs_list=[{'transform': ','.join(map(str, transforms_to_anchor[fn].flatten())),
                                'filename': fn + '.' + tb_fmt,
                                'output_fn': fn + '_thumbnail_alignedTo_' + anchor_fn + '.tif',
                                'pad_bg_color': 'black' if fn.split('-')[1][0] == 'F' else 'white'}
                                for fn in filenames],
                    exclude_nodes=[33],
                    argument_type='single',
                   cluster_size=12)
else:
    run_distributed('%(script)s %(stack)s %(input_dir)s %(out_dir)s %%(transform)s %%(filename)s %%(output_fn)s thumbnail 0 0 2000 1500 %%(pad_bg_color)s' % \
                    {'script_dir': script,
                    'stack': stack,
                    'input_dir': input_dir,
                    'out_dir': out_dir,
                    'pad_bg_color': pad_bg_color},
                    kwargs_list=[{'transform': ','.join(map(str, transforms_to_anchor[fn].flatten())),
                                'filename': fn + '.' + tb_fmt,
                                'output_fn': fn + '_thumbnail_alignedTo_' + anchor_fn + '.tif'}
                                for fn in filenames],
                    exclude_nodes=[33],
                    argument_type='single',
                   cluster_size=12)

print 'done in', time.time() - t, 'seconds'

Warping...
/shared/MouseBrainAtlas/preprocess/warp_crop_IM_v2.py MD589 /shared/CSHL_data/MD589 /shared/CSHL_data_processed/MD589/MD589_thumbnails_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091 0.974126730867,-0.226002460629,-432.928666649,0.226002460629,0.974126730867,-241.145665455,0.0,0.0,1.0 MD589-IHC82-2015.07.31-02.52.24_MD589_2_0245.tif MD589-IHC82-2015.07.31-02.52.24_MD589_2_0245_thumbnail_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091.tif thumbnail 0 0 2000 1500 white
done in 10.1449379921 seconds


qsub returned.


In [21]:
! tail ~/stderr.log

    custom_tf_fn = os.path.join(data_dir, stack, stack+'_custom_transforms', filenames[i] + '_to_' + filenames[i-1], filenames[i] + '_to_' + filenames[i-1] + '_customTransform.txt')
NameError: name 'data_dir' is not defined
MD589-IHC82-2015.07.31-02.52.24_MD589_2_0245_thumbnail_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091.tif
white
/bin/sh: 1: convert: not found
Child returned 127
MD589-IHC82-2015.07.31-02.52.24_MD589_2_0245_thumbnail_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091.tif
white
/bin/sh: 1: convert: not found
Child returned 127
